In [1]:
import spcm
from spcm import units
import numpy as np

# card : spcm.Card

ip = 'TCPIP::192.168.1.83::INSTR'

card = spcm.Card(ip)

card.open(ip)

# setup card for DDS
card.card_mode(spcm.SPC_REP_STD_DDS)

# Setup the channels
channels = spcm.Channels(card)
print(len(channels))
channels.enable(True)
channels.output_load(50 * units.ohm)
channels.amp(.428 * units.V)
card.write_setup()

# trigger mode
trigger = spcm.Trigger(card)
trigger.or_mask(spcm.SPC_TMASK_EXT0) # disable default software trigger
trigger.ext0_mode(spcm.SPC_TM_POS) # positive edge
trigger.ext0_level0(1.5 * units.V) # Trigger level is 1.5 V (1500 mV)
trigger.ext0_coupling(spcm.COUPLING_DC) # set DC coupling
card.write_setup()

# Setup DDS functionality
dds = spcm.DDS(card, channels=channels)
dds.reset()

dds.trg_src(spcm.SPCM_DDS_TRG_SRC_CARD)

core_list = [hex(2**n) for n in range(20)]

# core_list = [spcm.SPCM_DDS_CORE8,  spcm.SPCM_DDS_CORE9,  spcm.SPCM_DDS_CORE10, spcm.SPCM_DDS_CORE11, spcm.SPCM_DDS_CORE20]

# dds.cores_on_channel(1, *core_list)
card.start(spcm.M2CMD_CARD_ENABLETRIGGER)

2


In [ ]:
def split_array(f_split = 2.4e6, f_width_single_side = 4.e6, n_per_side = 4):
    f_center = 74.8e6
    # array1 = np.linspace( f_center-f_split/2-f_width_single_side ,f_center-f_split/2,n_per_side)
    array1 = f_center - f_split/2 - np.linspace( f_width_single_side , 0. ,n_per_side)
    # array2 = np.linspace( f_center+f_split/2+f_width_single_side ,f_center+f_split/2,n_per_side)
    array2 = f_center + f_split/2 + np.linspace( 0. , f_width_single_side ,n_per_side)
    # array1 = np.linspace(70.5e6,72.5e6,n)
    # array2 = np.linspace(77.5e6,79.5e6,n)
    return np.concatenate([array1,array2])

In [43]:
def parabolic_amps(flist):
    B = (1 - min(abs((flist - 75.e6)*1.e-6)**2) / max(abs((flist - 75.e6)*1.e-6)**2)) * len(flist)
    A = B/len(flist) / max(abs((flist - 75.e6)*1.e-6)**2)
    # A = 1.
    x = (flist - 75.e6)*1.e-6
    return A*x**2

# def parabolic_amps(flist):
#     B = 1.
#     A = B/len(flist) / max(abs((flist - 75.e6)*1.e-6)**2)
#     # A = 1.
#     x = (flist - 75.e6)*1.e-6
#     return A*x**2

In [45]:
f_list1 = np.linspace(76.e6,77.e3,5)
f_list2 = np.linspace(73.9e6,74.e3,2)
f_list = np.concatenate([f_list1,f_list2])

print(parabolic_amps(flist=f_list))
print(sum(parabolic_amps(flist=f_list)))

[1.78097383e-04 5.75801964e-02 2.43308181e-01 5.57362052e-01
 9.99741808e-01 2.15497833e-04 9.99821871e-01]
2.858207703095142


In [3]:
def compute_tweezer_phases(amplitudes):
        phases = np.zeros([len(amplitudes)])
        total_amp = np.sum(amplitudes)
        for tweezer_idx in range(len(amplitudes)):
            if tweezer_idx == 0:
                phases[0] =  360.
            else:
                phase_ij = 0
                for j in range(1,tweezer_idx):
                    phase_ij = phase_ij + 2*np.pi*(tweezer_idx - j)*(amplitudes[tweezer_idx] / total_amp)
                phase_i = (phase_ij % 2*np.pi) * 360
                phases[tweezer_idx] = phase_i
        return phases

# f_list1 = np.linspace(72.e6,74.e6,5)
# f_list2 = np.linspace(76.e6,78.e6,5)

# f_list = [74.1e6,76.0e6,73.2e6,78.3e6]

# f_list = [75.4e6,76.e6,76.6e6,77.2e6]
# f_list = [75.8e6]

# f_list = [74.e6,76.e6]

# f_list = np.linspace(75.8e6,78.e6,3)
# f_list2 = np.linspace(72.e6,74.e6,3)
# f_list = np.concatenate([f_list1,f_list2])

f_list = 75.e6 + np.linspace(-3.e6,3.e6,7)

# f_list = np.linspace(72.3e6,74.3e6,4)

n_tweezers = len(f_list)

# a_list1 = np.linspace(1/(2*(n_tweezers)),1/(2*(n_tweezers)),int(n_tweezers/2))
# a_list2 = 2*a_list1

# a_list = [.15,.15,.15,.15]
# a_list = [.5,.5]
# a_list = [.5,.5]
a_list = np.linspace(1/((n_tweezers)),1/((n_tweezers)),int(n_tweezers))
# a_list = [.2,.2,.2,.2]
# a_list = [.1,.2,.3,.4]

# a_list = np.array([.18,.21])

f_list = np.array(f_list)
a_list = np.array(a_list)

# idx = [0,1,2,3]
# a_list = a_list[idx]
# f_list = f_list[idx]

phases = compute_tweezer_phases(a_list)

# def normalize_alist(alist):
#     sum = np.sum(alist)
#     return alist/sum
# a_list = normalize_alist(a_list)

for tweezer_idx in range(len(core_list)):
    if tweezer_idx < len(f_list):
        dds[tweezer_idx].amp(a_list[tweezer_idx])
        dds[tweezer_idx].freq(f_list[tweezer_idx])
        dds[tweezer_idx].phase(phases[tweezer_idx])
    # else:
    #     dds[tweezer_idx].amp(0.)
    
dds.exec_at_trg()
dds.write_to_card()
trigger.force()

# Start command including enable of trigger engine

# input("Press Enter to Exit")

a_list

array([0.14285714, 0.14285714, 0.14285714, 0.14285714, 0.14285714,
       0.14285714, 0.14285714])

In [ ]:
# card.close(card._handle)
card.stop()
card.close(card._handle)